# OLD notebook
# Task 2

In [43]:
import numpy as np
import strid
import matplotlib.pyplot as plt
import scipy.signal
import functions as fun

In [44]:
#Perform covSSI on generated data to get the estimated modes
stochastic_data_path = "results/data-stochastic-8-floor.npz"
combined_data_path = "results/data-combined_8_floor.npz"
ssid, modes = fun.covSSI(stochastic_data_path)

In [45]:
%matplotlib nbagg
stabdiag = strid.StabilizationDiagram()
modes = stabdiag.plot(modes)

f, psd = ssid.psdy(nperseg=2**10)
stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)
#plt.savefig("figures/cov_SSI_output.jpg", dpi=300)

<IPython.core.display.Javascript object>

### Task 2.1 - finding relative difference

In [46]:
cluster_meat = fun.rel_difference(modes)
# !!! Have to handle modes in the lowest order!!!

In [47]:
print(np.max(cluster_meat))

0.9999898744335807


### Task 2.2 -  Using K-means clustering to separate all the poles into two groups

In [48]:
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(cluster_meat)
labels = kmeans.labels_

In [49]:
#Assign label to each mode
physical_coordinates = []
mathematical_coordinates = []
count = 0
physical_modes_dict = {}
physical_modes_list = []
num_modes = 0
for order in range(49, 5, -1):
    modes_of_order = modes[order]
    #print("Order: " + str(order))
    physical_modes_in_order = []
    for mode in modes_of_order:
        mode.physical = labels[count]
        if (mode.physical == 0):
            physical_modes_in_order.append(mode)
            physical_modes_list.append(mode)
            physical_coordinates.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])
        else:
            mathematical_coordinates.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])
        count += 1
    physical_modes_dict[order] = physical_modes_in_order
    num_modes += len(physical_modes_in_order)

physical_coordinates = np.array(physical_coordinates)
mathematical_coordinates = np.array(mathematical_coordinates)
physical_modes_list = np.array(physical_modes_list)


In [50]:
print(physical_modes_list.shape[0])

555


In [51]:
#Making a 3D scatterplot of frequency, damping and MAC number divided into physical and mathematical clusters
fig = plt.figure()
ax = plt.axes(projection='3d')

fg = ax.scatter3D(physical_coordinates[:,0], physical_coordinates[:,1], physical_coordinates[:,2], c="blue", label= "Physical modes")
ax.scatter3D(mathematical_coordinates[:,0], mathematical_coordinates[:,1], mathematical_coordinates[:,2], c="red", label="Mathematical modes")
ax.legend()
ax.set_xlabel("Frequency")
ax.set_ylabel("Damping")
ax.set_zlabel("MAC")
plt.savefig("figures/3dscatter.jpg", dpi=300)

print("Number of physical modes = " + str(physical_coordinates.shape[0]))
print("Number of mathematical modes = " +str(mathematical_coordinates.shape[0]))

<IPython.core.display.Javascript object>

Number of physical modes = 555
Number of mathematical modes = 52


In [52]:
%matplotlib nbagg
stabdiag = strid.StabilizationDiagram()
stabdiag.plot_partition_cluster_1(modes)

f, psd = ssid.psdy(nperseg=2**10)
stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)
#plt.savefig("figures/first_kmeans_output.jpg", dpi=300)

<IPython.core.display.Javascript object>

Length of color list: 302


# Task 3
### Task 3.1 Detecting structural modes by hierachical clustering
Using hierarchical clustering with a signle linkage method with a cut-off-distance of $d_c=0.04$
$$ d_{c_{i,j}} = d \lambda_{i,j} + (1- MAC_{i,j}) $$

In [53]:
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html
# Plotting dendogram: https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html
# https://www.youtube.com/watch?v=v7oLMvcxgFY&ab_channel=KindsonTheTechPro

import strid
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering


#Since we want to define our own distance between two poles, we must compute a distance matrix we can feed into the clustering algorithm.


In [54]:
distance = fun.distance_matrix(physical_modes_list)

/Users/emilneby/PyCharmProjects/TKT4550_Prosjekt/src/functions.py:177: ComplexWarning: Casting complex values to real discards the imaginary part
  dist_matrix[i, j] = (np.abs((eigen_i - eigen_j)) / np.max([eigen_i, eigen_j])) + (


Distance matrix computational time = 4.511608123779297sec


In [55]:
#Check the integrity of the distance matrix
print(np.max(distance))
print(np.mean(distance))
print(np.min(distance))
print(distance)

1.544131433929679
0.8809418823728999
0.0
[[0.         0.48364265 0.79178775 ... 0.25844957 0.95764903 0.98949459]
 [0.48364265 0.         0.49604142 ... 0.76526793 0.92498935 1.02580262]
 [0.79178775 0.49604142 0.         ... 0.91302412 0.93313836 0.86013469]
 ...
 [0.25844957 0.76526793 0.91302412 ... 0.         1.0178913  1.01152724]
 [0.95764903 0.92498935 0.93313836 ... 1.0178913  0.         1.00823237]
 [0.98949459 1.02580262 0.86013469 ... 1.01152724 1.00823237 0.        ]]


In [56]:
#Perform the actual clustering
model = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='single', distance_threshold=0.04)
#The agglomerative clustering algorithm only finds 1 non-structural mode with single linkage method. Works better so far with 'complete' or 'average'.

In [57]:
y_hc = model.fit_predict(distance)
model = model.fit(distance)
#print((y_hc))
print(np.max(y_hc))

90


In [58]:
fig = plt.figure(figsize=(25, 10))
dendogram = sch.dendrogram(sch.linkage(distance, method='single'))
plt.savefig("figures/dendogram.jpg", dpi=300)
plt.show()

<IPython.core.display.Javascript object>

/var/folders/xg/ddkjb2x96w167hqnv_gjv_v40000gn/T/ipykernel_79434/368536534.py:2: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  dendogram = sch.dendrogram(sch.linkage(distance, method='single'))


In [59]:
hierarchy = y_hc.tolist()
print(len(y_hc))
print(type(hierarchy))
#print(hierarchy)

num_modes_in_hierarchy = np.zeros((np.max(y_hc) + 1, 3))
for i in range(0, len(num_modes_in_hierarchy)):
    num_modes_in_hierarchy[i, 0] = hierarchy.count(i)
    num_modes_in_hierarchy[i, 1] = hierarchy.count(i)
    num_modes_in_hierarchy[i, 2] = i

print((num_modes_in_hierarchy))

555
<class 'list'>
[[18. 18.  0.]
 [ 4.  4.  1.]
 [ 9.  9.  2.]
 [ 9.  9.  3.]
 [ 6.  6.  4.]
 [ 4.  4.  5.]
 [39. 39.  6.]
 [ 2.  2.  7.]
 [ 3.  3.  8.]
 [ 5.  5.  9.]
 [ 7.  7. 10.]
 [ 2.  2. 11.]
 [ 4.  4. 12.]
 [ 2.  2. 13.]
 [ 3.  3. 14.]
 [13. 13. 15.]
 [ 6.  6. 16.]
 [69. 69. 17.]
 [ 3.  3. 18.]
 [ 2.  2. 19.]
 [ 2.  2. 20.]
 [ 2.  2. 21.]
 [47. 47. 22.]
 [ 2.  2. 23.]
 [ 1.  1. 24.]
 [33. 33. 25.]
 [ 5.  5. 26.]
 [ 2.  2. 27.]
 [ 4.  4. 28.]
 [ 1.  1. 29.]
 [ 2.  2. 30.]
 [ 1.  1. 31.]
 [ 2.  2. 32.]
 [ 3.  3. 33.]
 [ 3.  3. 34.]
 [ 3.  3. 35.]
 [ 2.  2. 36.]
 [ 2.  2. 37.]
 [ 1.  1. 38.]
 [ 1.  1. 39.]
 [ 1.  1. 40.]
 [ 1.  1. 41.]
 [ 1.  1. 42.]
 [ 2.  2. 43.]
 [ 5.  5. 44.]
 [ 1.  1. 45.]
 [ 4.  4. 46.]
 [ 1.  1. 47.]
 [44. 44. 48.]
 [ 1.  1. 49.]
 [ 1.  1. 50.]
 [ 1.  1. 51.]
 [ 1.  1. 52.]
 [43. 43. 53.]
 [ 1.  1. 54.]
 [ 1.  1. 55.]
 [ 1.  1. 56.]
 [ 1.  1. 57.]
 [ 1.  1. 58.]
 [ 1.  1. 59.]
 [ 1.  1. 60.]
 [ 1.  1. 61.]
 [ 2.  2. 62.]
 [ 1.  1. 63.]
 [ 1.  1. 64.]
 [ 1. 

In [60]:
structural_coordinates = []
non_structural_coordinates = []

for i in range(0, len(y_hc)):
    if y_hc[i] == 1:
        structural_coordinates.append(physical_coordinates[i,:])
    else:
        non_structural_coordinates.append(physical_coordinates[i,:])

structural_coordinates = np.array(structural_coordinates)
non_structural_coordinates = np.array(non_structural_coordinates)

In [61]:
#Assign hierarchy to each mode
count = 0
for order in range(49, 5, -1):
    modes_of_order = physical_modes_dict[order]
    for mode in modes_of_order:
        mode.cluster = hierarchy[count]
        count += 1

### Task 3.2 Need to visualize the hierarchical clusters

# Task 4
### Task 4.1 - Using k-means to divide into one cluster with "many modes" (structural modes) and one cluster with "few/scattered modes" (mathematcal modes).

In [62]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(num_modes_in_hierarchy[:,:2])
labels2 = kmeans.labels_

In [63]:
#print(labels2)
print("Number of clusters identified: " +str(len(labels2)))
#print((labels2))

Number of clusters identified: 91


In [64]:
# Pick the label with lowest occurence

labels2 = labels2.tolist()

structura_label = None
if (labels2.count(0) > labels2.count(1)):
    structural_label = 1
else:
    structural_label = 0



In [65]:
structural_hierarchies = []

for i in range(0, len(labels2)):
    if(labels2[i] == structural_label):
        structural_hierarchies.append(i)

print(structural_hierarchies)
#need to create a dictionary with colors to the clusters

[6, 17, 22, 25, 48, 53, 69, 71]


In [66]:
#Assign hierarchy to each mode
structural_modes_dict = {}
for order in range(49, 5, -1):
    modes_of_order = physical_modes_dict[order]
    structural_modes_in_order = []
    for mode in physical_modes_dict[order]:
        #print(mode.f)
        if mode.cluster in structural_hierarchies:
            mode.structural = 0
            structural_modes_in_order.append(mode)
        else:
            mode.structural = 1

    structural_modes_dict[order] = structural_modes_in_order

In [67]:
%matplotlib nbagg
stabdiag = strid.StabilizationDiagram()
stabdiag.plot_partition_cluster_2(physical_modes_dict)

f, psd = ssid.psdy(nperseg=2**10)
stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)
plt.savefig("figures/second_kmeans_output.jpg", dpi=300)

<IPython.core.display.Javascript object>

Length of color list: 302


In [68]:
%matplotlib nbagg
#Making a new stabilization diagram with the physical modes
stabdiag = strid.StabilizationDiagram()
stabdiag.plot_hierarchies(structural_modes_dict)

f, psd = ssid.psdy(nperseg=2**10)

stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)

<IPython.core.display.Javascript object>

Length of color list: 302


### Task 4.2 - Extract modal features of each detected mode as the average of all he components' features within each hierarchical cluster.

In [69]:

clustered_modes = fun.HierarchicalModes(stabdiag)

modes_in_clusters = clustered_modes.clusters_dict(structural_modes_dict)

In [70]:
#Verfying numbers of clusters up against stabilization diagram
#print(len(modes_in_clusters))
#clusters = (list(modes_in_clusters.values()))

In [71]:
from collections import defaultdict

hierarchies = sorted([*modes_in_clusters.keys()])
clusters = defaultdict(list)

for hierarchy in hierarchies:
    frequencies_in_hierarchy = []
    damping_in_hierarchy = []
    mode_shapes_in_hierarchy = []
    for element in modes_in_clusters[hierarchy]:
        frequencies_in_hierarchy.append(element.f)
        damping_in_hierarchy.append(element.xi)
        mode_shapes_in_hierarchy.append(element.v)

    f_mean = np.mean(np.array(frequencies_in_hierarchy))
    xi_mean = np.mean(np.array(damping_in_hierarchy))
    modal_shapes_mean = np.mean(np.array(mode_shapes_in_hierarchy), axis=0)

    clusters[hierarchy] = [f_mean, xi_mean, modal_shapes_mean]

clustered_features = dict(clusters)

In [72]:
#Sort the clusters by frequency to group into modes
sorted_features_dict = {cluster: mode for cluster, mode in sorted(clustered_features.items(), key=lambda item: item[1][0])}

sorted_features_list = np.array(list(sorted_features_dict.values()))

/var/folders/xg/ddkjb2x96w167hqnv_gjv_v40000gn/T/ipykernel_79434/3669789805.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sorted_features_list = np.array(list(sorted_features_dict.values()))


In [73]:
est_frequencies = sorted_features_list[:,0]
est_damping = sorted_features_list[:,1]
est_modeshapes = sorted_features_list[:,2]

In [74]:
print(est_frequencies)
print(est_damping)
print(est_modeshapes)

[2.9839954078233815 8.703524399883666 14.198546926802207 19.1886439531193
 23.733949236899697 27.105851832698043 29.768359333074095
 31.339131467936703]
[0.04742898228676199 0.02713637305380318 0.029902187556831748
 0.0330397188273515 0.04601905125891591 0.04099844210034399
 0.05515848997241481 0.049771087040230755]
[array([ 0.0011831 +0.01996592j, -0.00405328+0.03681565j,
        -0.00406428+0.04931858j, -0.01441917+0.06878751j,
        -0.01012428+0.08337107j, -0.01630645+0.08440667j,
        -0.0102961 +0.0988222j , -0.01935364+0.10556655j])
 array([-0.00673781+0.02867324j, -0.01294452+0.04802217j,
        -0.01364059+0.05509876j, -0.01209512+0.04399937j,
        -0.00451941+0.02094275j,  0.00272291-0.01058962j,
         0.00918491-0.03643091j,  0.01275312-0.05220362j])
 array([ 5.01832464e-05+2.08794281e-04j, -1.60243366e-04+1.15436264e-04j,
         1.25882201e-04-2.03187235e-04j,  6.16399404e-05+1.09193436e-04j,
        -1.06639427e-04+5.51898126e-05j,  1.18451422e-04+2.04311400e

### Task 4.3 - Verify against ground truth

In [75]:
data = np.load("results/data-stochastic-8-floor.npz")

true_f = data["true_frequencies"]
true_xi = data["true_damping"]
true_modeshapes = data["true_modeshapes"]

In [76]:
print(true_f)

[ 2.9369931   8.71096352 14.18829253 19.18245625 23.52338443 27.06325197
 29.68151292 31.28900555]


In [77]:
print(np.mean(np.abs(true_f-est_frequencies)))

0.057627565857315766


In [83]:
fig = plt.figure("Damping estimate")
axd = fig.add_axes((0.1, 0.1, .8, .8))
axd.set(xlabel='Frequency', ylabel='Damping ratio', title='Estimated and true frequency and damping',
        ylim=(0, .10)
       )
figmodes, axes = plt.subplots(ncols=4, nrows=2, dpi=144)
res = []
for n in range(true_f.size):
    ax = axes.flatten()[n]
    un = true_modeshapes[n]
    fn = true_f[n]
    xin = true_xi[n]
    #nmax = np.argmax([strid.modal_assurance_criterion(mode.v, un) for mode in modes])
    #mode = modes[nmax]

    line, = axd.plot(est_frequencies[n], est_damping[n], 'x')
    line, = axd.plot(fn, xin, 'o', mec=line.get_color(), mfc=(0, 0, 0, 0))


    ferr = (est_frequencies[n]-fn)/fn * 100
    xierr = (est_damping[n]-xin)/xin*100
    mac = strid.modal_assurance_criterion(un, est_modeshapes[n])
    res.append([n, ferr, xierr, mac*100,])


    v_true = np.r_[0., un]
    v = np.r_[0, est_modeshapes[n]]
    v = strid.modal_scale_factor(v, v_true)*v
    z = np.arange(v.size)
    ax.plot(v_true, z, label='True')
    ax.plot(v.real, z, label='Estimated')

    if n == 2:
        ax.legend(bbox_to_anchor=(.5, 1.1), loc='lower center', ncol=2)
        axd.legend(['Estimated', 'True'],)
    ax.axvline(0., color=(0, 0, 0, .3))

    ax.set_title(f"Mode {n}")
    ax.axis('off')
    ax.set_xlim(-.5, .5)

plt.savefig("figures/accuracy.jpg", dpi=300)

<IPython.core.display.Javascript object>